# Agents to parse and comment an income statement

This time we will try and use agents to download, parse, and comment on an income statement.

The document is real, and it is publicly available on the internet at the time of writing.

# Setup: packages and environment variables

In [ ]:
import importlib
import os

if not importlib.util.find_spec("class_utils"):
    !pip install -qqq git+https://github.com/xtreamsrl/genai-for-engineers-class

In [ ]:
import autogen

os.environ["OPENAI_API_KEY"] = ...

# Creating agents

For this purpose, we will use the latest model, GPT-4o.

Less advanced models are less effective in writing code.

In [ ]:
llm_config = {
    "config_list": [{"model": "gpt-4o"}],
}

# User Proxy
UserProxyAgent is a special type of agent that can be used to proxy user input to another agent or group of agents. It supports the following human input modes:
- ALWAYS: Always ask user for input.
- NEVER: Never ask user for input. In this mode, the agent will use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.
- AUTO: Only ask user for input when conversation is terminated by the other agent(s). Otherwise, use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.

We will execute the code natively without docker. This is a security flaw, and in real life it should be avoided, as the execution of arbitrary code can expose sensitive data.

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "income_statement_analysis",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

# Define roles

It's your turn now. Define the agents you want to use.

In [ ]:
group_chat = autogen.GroupChat(
    agents=[user_proxy], messages=[], max_round=30, send_introductions=True
)

manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

# Run the chat
Let's trigger the grup chat and see what happens.

In [ ]:
link = "https://www.cisiaonline.it/_mamawp/wp-content/uploads/2015/07/Bilancio-esercizio_Nota-integr_CISIA-2017.pdf"

user_proxy.initiate_chat(
    manager,
    message=f"At this link is an income statement: {link}. "
    "Please evaluate it, considering it is written in Italian.",
)